In [0]:
pip install -q requests beautifulsoup4 

In [0]:
dbutils.library.restartPython()

In [0]:
import requests
from bs4 import BeautifulSoup
import time
import json
from pyspark.sql.functions import col, current_timestamp, timestamp_diff

In [0]:
def get_soup(url):
    page_to_scrape = requests.get(url)
    soup = BeautifulSoup(page_to_scrape.text, "html.parser")
    return soup

In [0]:
def get_job_details(url):
    soup = get_soup(url)
    d = json.loads(soup.find("script", attrs={"type": "application/ld+json"}).text)
    soup1 = BeautifulSoup(d['description'], "html.parser")
    x = (soup1.get_text(separator=' ', strip=True))
    soup2 = BeautifulSoup(x, "html.parser")
    d['description'] =(soup2.get_text(separator=' ', strip=True))
    fields = ['title', 'description', 'datePosted', 'skills']
    selected = {k:v for k,v in d.items() if k in fields}
    selected['url'] = url
    selected['company'] = d['hiringOrganization']['name']
    selected['location'] = d['jobLocation']['address']['addressLocality']
    return selected

In [0]:
url = "https://www.linkedin.com/jobs/search?keywords=Data+Engineer&location=India&f_TPR=r86400&f_E=2%2C3"

soup = get_soup(url)
data = soup.findAll("a", attrs={"class":"base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2] outline-offset-[4px]"})

In [0]:
details = []
for job in data:
    time.sleep(1)
    try:
        details.append(get_job_details(job['href']))
    except:
        pass
print(f"{len(data)-len(details)} jobs skipped")

In [0]:
df = spark.createDataFrame(details)
df = df.withColumn("hrs_ago", timestamp_diff("hour",  col("datePosted").cast("timestamp"), current_timestamp()))
df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("raw_catalogue.jobs.linkedJobs")

In [0]:
display(df)